# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

Here is an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

# Library Imports
We need to do is import the open-source software libraries that we'll be using.

In [17]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import math #The Python math module

## Importing list of stocks
Need to import list of stocks and API token before proceeding.

In [18]:
#using IE X cloud API for getting data
stocks = pd.read_csv('sp_500_stocks.csv')
#saved token in secrets.py, for confidentiality
from secrets import IEX_CLOUD_API_TOKEN

## Making API Call
Need to get one-year price returns for each stock of S&P 500.

In [19]:
#checking what all key value pairs are generated 
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2183793084836,
 'week52high': 145.06,
 'week52low': 78.55,
 'week52highSplitAdjustOnly': 148.45,
 'week52lowSplitAdjustOnly': 80.37,
 'week52change': 0.6958004049089623,
 'sharesOutstanding': 17263889829,
 'float': 0,
 'avg10Volume': 101921186,
 'avg30Volume': 97542857,
 'day200MovingAvg': 130.85,
 'day50MovingAvg': 130.56,
 'employees': 153903,
 'ttmEPS': 4.64,
 'ttmDividendRate': 0.8376255208234226,
 'dividendYield': 0.006787166256707953,
 'nextDividendDate': '0',
 'exDividendDate': '2021-05-02',
 'nextEarningsDate': '2021-07-20',
 'peRatio': 28.68335279543827,
 'beta': 1.4197115959475652,
 'maxChangePercent': 50.06640486510111,
 'year5ChangePercent': 5.176583583693276,
 'year2ChangePercent': 1.80035376758367,
 'year1ChangePercent': 0.6833605413630766,
 'ytdChangePercent': -0.03656606280663307,
 'month6ChangePercent': 0.07292913993740216,
 'month3ChangePercent': -0.0587539243962316,
 'month1ChangePercent': -0.0508340050648444,
 'day30ChangeP

## Executing A Batch API Call & Building Our DataFrame
Time to execute several batch API calls and add the information we need to our DataFrame.

Start by running the following code cell. It contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [36]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.


In [43]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,135.92,0.623309,N/A
1,AAL,22.40,1.49482,N/A
2,AAP,211.89,0.635364,N/A
3,AAPL,133.23,0.685203,N/A
4,ABBV,121.61,0.354233,N/A
...,...,...,...,...
500,YUM,119.82,0.439275,N/A
501,ZBH,171.60,0.50837,N/A
502,ZBRA,506.37,1.26862,N/A
503,ZION,61.90,1.31602,N/A


## Removing Low-Momentum Stocks
The investment strategy seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [41]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,69.69,5.42332,0
1,FCX,42.31,4.00899,0
2,GPS,36.60,3.91871,0
3,IVZ,28.78,3.34506,0
4,TPR,48.11,2.96638,0
5,MOS,37.42,2.67258,0
6,KSS,63.81,2.60909,0
7,SIVB,606.40,2.37582,0
8,DFS,118.03,2.21162,0
9,PWR,98.99,2.1572,0


## Calculating the Number of Shares to Buy


In [40]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
portfolio_size

Enter the value of your portfolio:1000000


'1000000'

In [28]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe


C:\Users\hp\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,69.69,5.42332,0
1,FCX,42.31,4.00899,0
2,GPS,36.60,3.91871,0
3,IVZ,28.78,3.34506,0
4,TPR,48.11,2.96638,0
5,MOS,37.42,2.67258,0
6,KSS,63.81,2.60909,0
7,SIVB,606.40,2.37582,0
8,DFS,118.03,2.21162,0
9,PWR,98.99,2.1572,0
